In [ ]:
import os
import numpy as np
import pandas as pd

import glob
import copy

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_expert_d = []
for file in glob.glob('./expert_direction_seed/results/expert*/rewards.npz'):
    ev = np.load(file)
    rew =  ev['results'][:25]
    data_expert_d.append(pd.DataFrame({'timesteps': (np.arange(0, len(rew), 1)+1)*16384,
                                       'reward': rew,
                                       'level': 'expert'}))
    
# data_expert_d_sde = []
# for file in glob.glob('./expert_direction_seed_sde/results/expert*/rewards.npz'):
#     ev = np.load(file)
#     data_expert_d_sde.append(pd.DataFrame({'timesteps': (np.arange(0, len(ev['results']), 1)+1)*16384,
#                                        'value': ev['results'],
#                                        'level': 'expert'}))    
    
data_expert_xy = []    
for file in glob.glob('./expert_xy_seed/results/expert*/rewards.npz'):
    ev = np.load(file)
    rew =  ev['results'][:25]
    data_expert_xy.append(pd.DataFrame({'timesteps': (np.arange(0, len(rew), 1)+1)*16384,
                                       'reward': rew,
                                       'level': 'expert'}))
    
data_noob = []    
for file in glob.glob('./noob_seed/results/noob*/rewards.npz'):
    ev = np.load(file)
    rew =  ev['results'][:25]
    data_noob.append(pd.DataFrame({'timesteps': (np.arange(0, len(rew), 1)+1)*16384,
                                       'reward': rew,
                                       'level': 'expert'}))

data_noob512128 = []    
for file in glob.glob('./noob_seed_512128/results/noob*/rewards.npz'):
    ev = np.load(file)
    rew =  ev['results'][:25]
    data_noob512128.append(pd.DataFrame({'timesteps': (np.arange(0, len(rew), 1)+1)*16384,
                                       'reward': rew,
                                       'level': 'expert'}))
        


In [ ]:
# data_expert = []
# for file in glob.glob(log_prefix + 'results/expert*/evaluations.npz'):
#     ev = np.load(file)
#     for i in range(5):
#         data_expert.append(pd.DataFrame({'timesteps': ev['timesteps'],
#                                        'value': ev['results'][:, i],
#                                        'level': 'expert'}))
    
# data_noob= []
# for file in glob.glob(log_prefix +'results/noob*/evaluations.npz'):
#     ev = np.load(file)
#     for i in range(5):
#         data_noob.append(pd.DataFrame({'timesteps': ev['timesteps'],
#                                    'value': ev['results'][:, i],
#                                    'level': 'noob'}))

In [ ]:
fig = plt.figure()
plot_data(data_expert_xy, smooth=3, label='xy')
plot_data(data_expert_d, smooth=3, label='dir')
plot_data(data_noob, smooth=3, label='orig')
# plot_data(data_noob512128, smooth=3, label='orig_obs_n512')
# plt.savefig('Pictures/CarRacing_obs_comp.pdf', bbox_inches='tight')
# fig.legend(mode='expand', loc='upper center', ncol=4, frameon=False, prop={'size': 13})
# fig.legend(
#     loc='outside upper center',
#     ncol=4,
#     handlelength=1.0, 
#     # mode="expand", 
#     borderaxespad=0.,
#     frameon=False,
#     prop={'size': 13}
# )
plt.xlim(0, 4*1e5)
plt.show()

In [ ]:
def plot_data(data, xaxis='timesteps', value="reward", smooth=1, **kwargs):
    data = copy.deepcopy(data)
    if smooth > 1:
        """
        smooth data with moving window average.
        that is,
            smoothed_y[t] = average(y[t-k], y[t-k+1], ..., y[t+k-1], y[t+k])
        where the "smooth" param is width of that window (2k+1)
        """
        y = np.ones(smooth)
        for datum in data:
            x = np.asarray(datum[value])
            z = np.ones(len(x))
            smoothed_x = np.convolve(x,y,'same') / np.convolve(z,y,'same')
            datum[value] = smoothed_x

    if isinstance(data, list):
        data = pd.concat(data, ignore_index=True)
    sns.set(style="whitegrid", font_scale=1.5) # darkgrid whitegrid
    sns.lineplot(data=data, x=xaxis, y=value, errorbar='sd', **kwargs, legend=None)
    plt.legend(
            # loc='upper center',
            # bbox_to_anchor=(0.5, 1.0),
            bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
            ncol=4,
            handlelength=1.0, 
            mode="expand", 
            borderaxespad=0.,
            frameon=False,
            prop={'size': 13}
    )

    # plt.legend(loc='best').set_draggable(True)
    #plt.legend(loc='upper center', ncol=3, handlelength=1,
    #           borderaxespad=0., prop={'size': 13})

    """
    For the version of the legend used in the Spinning Up benchmarking page, 
    swap L38 with:
    plt.legend(loc='upper center', ncol=6, handlelength=1,
               mode="expand", borderaxespad=0., prop={'size': 13})
    """

    xscale = np.max(np.asarray(data[xaxis])) > 5e3
    if xscale:
        # Just some formatting niceness: x-axis scale in scientific notation if max x is large
        plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.tight_layout(pad=2)